In [ ]:
import cv2
import numpy as np

# the purpose of this file is creating a dataset of images consist of only ears. Original images consist of full bodies
def loadtocrop(test):

    if(test):
        size = 105  # test size is 105
    else:
        size = 500 # training size is 500

    ctr = -1 # for file numbers

    for shp in range(0):
        for i in range(size):

            ctr += 1

            if(test):
                o_landmark_path = 'data/test/o_landmarks/test_' + str(i) + '.txt'       # original image path
                o_image_path = 'data/test/o_images/test_' + str(i) + '.png'             # original landmark path
                landmark_path = 'data/test/landmarks/test_' + str(ctr) + '.txt'         # preprocessed image path
                image_path = 'data/test/images/test_' + str(ctr) + '.png'               # preprocessed landmark path
            else:
                o_landmark_path = 'data/train/o_landmarks/train_' + str(i) + '.txt'     # original image path
                o_image_path = 'data/train/o_images/train_' + str(i) + '.png'           # original landmark path
                landmark_path = 'data/train/landmarks/train_' + str(ctr) + '.txt'       # preprocessed image path
                image_path = 'data/train/images/train_' + str(ctr) + '.png'             # preprocessed landmark path

            smallest_x = 999999
            smallest_y = 999999
            greatest_x = 0
            greatest_y = 0

            img = cv2.imread(o_image_path)          # take the original image


            with open(o_landmark_path, 'r') as f:
                lines_list = f.readlines()         # all lines as list

                for j in range(3, 58):          # in landmark text files, landmarks start at 3rd line end in 57th
                    string = lines_list[j]
                    str1, str2 = string.split(' ')
                    x_ = float(str1)
                    x_ = round(x_, 3)
                    y_ = float(str2)
                    y_ = round(y_, 3)


                    if(x_ < smallest_x):
                        smallest_x = int(x_)
                    if(x_ > greatest_x):
                        greatest_x = int(x_)
                    if (y_ < smallest_y):
                        smallest_y = int(y_)
                    if (y_ > greatest_y):
                        greatest_y = int(y_)

                    if (j == 3):                # if first landmark point
                        temp_x = np.array(x_)
                        temp_y = np.array(y_)

                    else:                           # if not first landmark point
                        temp_x = np.hstack((temp_x, x_))
                        temp_y = np.hstack((temp_y, y_))

            # some padding
            if(smallest_x>5 and smallest_y>5):
                smallest_x -= 5
                smallest_y -= 5
            else:
                smallest_x = 0
                smallest_y = 0
            greatest_x += 5
            greatest_y += 5

            for k in range(55):     # adjustment to the cropped image
                temp_x[k] -= smallest_x
                temp_y[k] -= smallest_y

            width = greatest_x - smallest_x
            height = greatest_y - smallest_y

            for k in range(55):     # normalization
                temp_x[k] = (temp_x[k] / width)
                temp_y[k] = (temp_y[k] / height)

                if (shp == 1): #flip vertical
                    temp_x[k] = 1 - temp_x[k]

                if(shp == 2): #rotate left
                    wow = temp_x[k]
                    temp_x[k] = temp_y[k]
                    temp_y[k] = wow
                    temp_y[k] = 1 - temp_y[k]

                if (shp == 3):  # rotate right
                    wow = temp_x[k]
                    temp_x[k] = temp_y[k]
                    temp_y[k] = wow
                    temp_x[k] = 1 - temp_x[k]

                if (shp == 4):  # flip vertical + rotate left
                    temp_x[k] = 1 - temp_x[k]
                    wow = temp_x[k]
                    temp_x[k] = temp_y[k]
                    temp_y[k] = wow
                    temp_y[k] = 1 - temp_y[k]

                if (shp == 5):  # flip vertical + rotate right
                    temp_x[k] = 1 - temp_x[k]
                    wow = temp_x[k]
                    temp_x[k] = temp_y[k]
                    temp_y[k] = wow
                    temp_x[k] = 1 - temp_x[k]


            if (i == 0):        # if first image's landmarks
                Y = np.hstack((temp_x, temp_y))
                Y = Y[None, :]

            else:               # if not first image's landmarks
                temp = np.hstack((temp_x, temp_y))
                temp = temp[None, :]
                Y = np.vstack((Y, temp))

            # crop and resize to desired dimensions
            cv2.rectangle(img, (smallest_x, smallest_y), (greatest_x, greatest_y), (255, 255, 255), 3)
            crop_image = img[smallest_y:greatest_y, smallest_x:greatest_x]
            resize_image = cv2.resize(crop_image, (224, 224))

            if(shp == 1):           # flip vertical
                resize_image = cv2.flip(resize_image, 1)

            if(shp == 2):           # rotate left
                resize_image = np.rot90(resize_image)

            if(shp == 3):           # rotate right
                resize_image = np.rot90(resize_image,3)

            if (shp == 4):          # flip vertical + rotate left
                resize_image = cv2.flip(resize_image, 1)
                resize_image = np.rot90(resize_image)

            if (shp == 5):          # flip vertical + rotate right
                resize_image = cv2.flip(resize_image, 1)
                resize_image = np.rot90(resize_image, 3)

            # same the image
            cv2.imwrite(image_path, resize_image)


            # save the landmarks
            with open(landmark_path, 'w') as f:
                f.write("version: 2\n")
                f.write("n_points: 55\n")
                f.write("{\n")
                for p in range(55):
                    line = str(Y[i][p]) + " " + str(Y[i][p + 55]) + "\n"
                    f.write(line)
                f.write("}\n")
                f.close()




In [1]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [2]:
!unzip gdrive/My\ Drive/data.zip

Archive:  gdrive/My Drive/data.zip
   creating: data/single/
   creating: data/single/result/
  inflating: data/single/result/result.png  
  inflating: data/single/sibjfkf.png  
  inflating: data/single/single_img.png  
   creating: data/test/
   creating: data/test/images/
  inflating: data/test/images/test_0.png  
  inflating: data/test/images/test_1.png  
  inflating: data/test/images/test_10.png  
  inflating: data/test/images/test_100.png  
  inflating: data/test/images/test_101.png  
  inflating: data/test/images/test_102.png  
  inflating: data/test/images/test_103.png  
  inflating: data/test/images/test_104.png  
  inflating: data/test/images/test_11.png  
  inflating: data/test/images/test_12.png  
  inflating: data/test/images/test_13.png  
  inflating: data/test/images/test_14.png  
  inflating: data/test/images/test_15.png  
  inflating: data/test/images/test_16.png  
  inflating: data/test/images/test_17.png  
  inflating: data/test/images/test_18.png  
  inflating: data/

In [ ]:
loadtocrop(test=False)
loadtocrop(test=True)

In [ ]:
from keras.models import Sequential
import tensorflow as tf
from keras.models import load_model
from keras.layers import Conv2D, MaxPooling2D, Dropout, RepeatVector
from keras.layers import BatchNormalization
from keras.layers import Flatten, Dense

def get_my_CNN_model_architecture():

    model = Sequential()
    model.add(Conv2D(16, (3, 3), input_shape=(224, 224, 3), kernel_initializer='random_uniform', activation='relu'))

    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.3))

    model.add(Conv2D(256, (5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(512, (5, 5), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Flatten())

    model.add(Dense(1024, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.7))

    model.add(Dense(110))

    return model

def compile_model(model, optimizer, loss, metrics):
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

def train_model(model, X_train, Y_train, epochs, batch_size):
    return model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, verbose=1, validation_split=0.2)

def save_model(model, fileName):
    model.save(fileName + '.h5')

def load_current_model(fileName):
    return load_model(fileName + '.h5')

def test_model(model, X_test, Y_test):
    preds = model.evaluate(X_test, Y_test)
    print("Loss = " + str(preds[0]))
    print("Test Accuracy = " + str(preds[1]))

def summarize_model(model):
    model.summary()



In [ ]:
import cv2
import numpy as np
import tensorflow as tf
import keras.utils as image
from keras.applications.imagenet_utils import preprocess_input

single_img=False
def load_data(test=False, size=500, test_size=105, single_img=single_img, single_img_path='data/single/single_img.png'):


    if (test):
        size = test_size

    if(single_img):
        size = 1

    for i in range(0, size):        # take the images

        img_path = 'data/train/images/train_' + str(i) + '.png'
        if (test):
            img_path =  'data/test/images/test_' + str(i) + '.png'

        if (single_img):

            img_path = single_img_path
            img = cv2.imread(img_path)
            img = cv2.resize(img, (224, 224)) # first resize the original single image
            cv2.imwrite('data/single/single_img.png',img)       # save the resized original single image
            img = image.load_img('data/single/single_img.png')      # load the resized original single image
            x = image.img_to_array(img)
            x = np.expand_dims(x, axis=0)
            x = preprocess_input(x)
            y = None
            return x, y         # return the single image as preprocessed for the model

        img = image.load_img(img_path)
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        if (i == 0):            # if first image
            X = x
            continue
        X = np.vstack((X, x))       # if not first image



    for i in range(0, size):        # take the landmarks


        txt_path = 'data/train/landmarks/train_' + str(i) + '.txt'
        if (test):
            txt_path = 'data/test/landmarks/test_' + str(i) + '.txt'



        with open(txt_path, 'r') as f:
            lines_list = f.readlines()

            for j in range(3, 58):      # in landmark text files, landmarks start at 3rd line end in 57th
                string = lines_list[j]
                str1, str2 = string.split(' ')
                x_ = float(str1)
                y_ = float(str2)
                if (j == 3):            # if first landmark point
                    temp_x = np.array(x_)
                    temp_y = np.array(y_)
                    continue 

                # if not first landmark point

                temp_x = np.hstack((temp_x, x_))
                temp_y = np.hstack((temp_y, y_))


        if (i == 0):            # if first image's landmarks
            Y = np.hstack((temp_x, temp_y))
            Y = Y[None, :]
            continue

        # if not first image's landmarks
        temp = np.hstack((temp_x, temp_y))
        temp = temp[None, :]
        Y = np.vstack((Y, temp))

    return X, Y


In [ ]:
from keras.optimizers import SGD, Adam
import numpy as np

# Load
X_train, Y_train = load_data(size=500,single_img=False)

# Shuffle
np.random.seed(142)
np.random.shuffle(X_train)
np.random.seed(142)
np.random.shuffle(Y_train)

# Architecture
my_model = get_my_CNN_model_architecture()

# adam optimizer
adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

# Compile
compile_model(my_model, optimizer = adam, loss = 'mean_squared_error', metrics=['accuracy'])

# Train
hist = train_model(my_model, X_train, Y_train, epochs=300, batch_size=32)

# Save
save_model(my_model, 'my_model')

# Summary
summarize_model(my_model)


/usr/local/lib/python3.9/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/300
13/13 [==============================] - 15s 291ms/step - loss: 5.4958 - accuracy: 0.0100 - val_loss: 3916.9836 - val_accuracy: 0.0000e+00
Epoch 2/300
13/13 [==============================] - 2s 128ms/step - loss: 4.3333 - accuracy: 0.0150 - val_loss: 470.2849 - val_accuracy: 0.0000e+00
Epoch 3/300
13/13 [==============================] - 2s 131ms/step - loss: 3.8466 - accuracy: 0.0025 - val_loss: 75.9148 - val_accuracy: 0.0000e+00
Epoch 4/300
13/13 [==============================] - 2s 127ms/step - loss: 3.4786 - accuracy: 0.0175 - val_loss: 33.2043 - val_accuracy: 0.0000e+00
Epoch 5/300
13/13 [==============================] - 2s 131ms/step - loss: 3.2233 - accuracy: 0.0200 - val_loss: 9.5333 - val_accuracy: 0.0000e+00
Epoch 6/300
13/13 [==============================] - 2s 132ms/step - loss: 2.9871 - accuracy: 0.0025 - val_loss: 2.1161 - val_accuracy: 0.0000e+00
Epoch 7/300
13/13 [==============================] - 2s 136ms/step - loss: 2.7316 - accuracy: 0.0100 - val_los

In [2]:
#COPY
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [4]:
# result corresponds to result of machine trained with 500 images


import matplotlib.pyplot as plt

def put_landmarks(i, j,pred, single_img):
    # print (j)
    if(single_img):      # if the case is single sample, not a whole set
        img_path = '/content/gdrive/MyDrive/input_img/single_img'+str(j)+'.png'
        img_result_path = '/content/gdrive/MyDrive/output_img/result'+str(j)+'.png'

    img_original = plt.imread(img_path)

    for k in range(0,55):  # drop the landmark points on the image
        plt.scatter([pred[k]], [pred[k+55]])


    plt.imshow(img_original)
    plt.savefig(img_result_path)
    plt.close()


In [ ]:
# original_result corresponds to result of machine trained with 3000 images

import matplotlib.pyplot as plt

def put_landmarks1(i,j, pred, single_img):
    print (j)
    if(single_img):      # if the case is single sample, not a whole set
        img_path = '/content/gdrive/MyDrive/input_img/single_img'+str(j)+'.png'
        img_result_path = '/content/gdrive/MyDrive/output_img/original_result'+str(j)+'.png'

    img_original = plt.imread(img_path)

    for k in range(0,55):  # drop the landmark points on the image
        plt.scatter([pred[k]], [pred[k+55]])


    plt.imshow(img_original)
    plt.savefig(img_result_path)
    plt.close()

In [ ]:
import numpy as np
import tensorflow as tf
import keras.utils as image
from keras.applications.imagenet_utils import preprocess_input
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Conv2D, MaxPooling2D, Dropout, RepeatVector
from keras.layers import BatchNormalization
from keras.layers import Flatten, Dense
import os
import cv2
lst = os.listdir('/content/gdrive/MyDrive/input_img') # your directory path
number_files = len(lst)+1
print ("#")
print (number_files)
model = load_model('/content/gdrive/MyDrive/my_model.h5')
for j in range(1,number_files):
    img_path = '/content/gdrive/MyDrive/input_img/single_img'+str(j)+'.png'
    img = cv2.imread(img_path)
    img = cv2.resize(img, (224, 224)) # first resize the original single image
    cv2.imwrite('/content/gdrive/MyDrive/input_img/single_img'+str(j)+'.png',img)       # save the resized original single image
    img = image.load_img('/content/gdrive/MyDrive/input_img/single_img'+str(j)+'.png')      # load the resized original single image
    X = image.img_to_array(img)
    X = np.expand_dims(X, axis=0)
    X = preprocess_input(X)
    Y = None
    for i in range(0,len(X)):

      temp = X[i]
      temp = temp[None,:] # adjust the dimensions for the model
      prediction = model.predict(temp)

      for p in range(len(prediction[0])):     # adjust the landmark points for 224x224 image (they were normalized in range 0 to 1)

          prediction[0][p] = int(prediction[0][p] * 224)
      # print(j)
      put_landmarks(i,j, prediction[0], single_img=True)        # the function to drop landmark points on the image

In [ ]:
from keras.models import Sequential
import tensorflow as tf
from keras.models import load_model
from keras.layers import Conv2D, MaxPooling2D, Dropout, RepeatVector
from keras.layers import BatchNormalization
from keras.layers import Flatten, Dense
import os
lst = os.listdir('/content/gdrive/MyDrive/input_img') # your directory path
number_files = len(lst)+1
model = load_model('/content/gdrive/MyDrive/my_model1.h5')
for j in range(1,number_files):
    img_path = '/content/gdrive/MyDrive/input_img/single_img'+str(j)+'.png'
    img = cv2.imread(img_path)
    img = cv2.resize(img, (224, 224)) # first resize the original single image
    cv2.imwrite('/content/gdrive/MyDrive/input_img/single_img'+str(j)+'.png',img)       # save the resized original single image
    img = image.load_img('/content/gdrive/MyDrive/input_img/single_img'+str(j)+'.png')      # load the resized original single image
    X = image.img_to_array(img)
    X = np.expand_dims(X, axis=0)
    X = preprocess_input(X)
    Y = None
    for i in range(0,len(X)):

      temp = X[i]
      temp = temp[None,:] # adjust the dimensions for the model
      prediction = model.predict(temp)

      for p in range(len(prediction[0])):     # adjust the landmark points for 224x224 image (they were normalized in range 0 to 1)

          prediction[0][p] = int(prediction[0][p] * 224)

      put_landmarks1(i,j, prediction[0], single_img=True)        # the function to drop landmark points on the image

1/1 [==============================] - 0s 127ms/step
1
1/1 [==============================] - 0s 21ms/step
2
1/1 [==============================] - 0s 33ms/step
3
1/1 [==============================] - 0s 30ms/step
4
1/1 [==============================] - 0s 38ms/step
5
1/1 [==============================] - 0s 33ms/step
6
1/1 [==============================] - 0s 21ms/step
7
